In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear, Sigmoid, ReLU
from torch.autograd import Variable

In [ ]:
X_arr=[]
Y_arr=[]
#это пример с рандомными значениями
# Предположим нам надо предсказывать плотность загрузки по средним размерам коробки в контейнере
folder_path = "C:/Users/AliSa/Prakticum/ALGORITM"
directory_contents = os.listdir(folder_path)
for filename in directory_contents:
    df = pd.pandas.read_json("C://Users//AliSa//Prakticum//ALGORITM//"+filename)
    data = pd.DataFrame(df['data'])['data']
    cargoes = pd.DataFrame(list(data[0]))
    sizes = pd.DataFrame(list(cargoes['size']))
    sizes = sizes.iloc[:].values
    avg_width=0
    avg_height=0
    avg_length=0
    i=0
    for size in sizes:
        avg_width += size[0]
        avg_height += size[1]
        avg_length += size[2]
        i+=1
    avg_width /= i
    avg_height /= i
    avg_length /= i
    avg_all = (avg_width+avg_height+avg_length)/i
    avg_all = 1/avg_all
    X_arr.append([avg_width,avg_height,avg_length])
    data_result = pd.DataFrame(df['data_result'])['data_result']
    cargo_space = data_result['cargo_space']
    calculation_info = cargo_space['calculation_info']['density_percent']
    Y_arr.append(calculation_info)
X_arr= np.vstack(X_arr)
X_arr
print(Y_arr)
print(X_arr)

In [ ]:
# В библиотеке torch для обучения данные должны быть в формате тензоров
# соответственно разделяем
# затем приводим к формату
X = torch.autograd.Variable(torch.FloatTensor(X_arr))
Y = torch.autograd.Variable(torch.FloatTensor(Y_arr))
# количество признаков потом указывает на вход в нейроне
num_features = X.shape[1]

In [ ]:
# Создаем сеть то есть Sequential - последовательность
# сюда внутрь можно добавлять разные слои
# документация - https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
model = torch.nn.Sequential(
    Linear(num_features, 1),
    ReLU()
)

# Пока здесь один слой линейный - Linear
# ReLU() - функция активация, после слоя
# функция активация тоже могут быть разные - можно погулить

In [ ]:
# задаем какую-нибудь функцию потерь для обучения
# здесь стандартная как для регрессии - квадратичная ошибка
loss_fn = torch.nn.MSELoss(size_average=False)

# шаг градиентного спуска (точнее -- метода оптимизации)
learning_rate = 0.00001  
# сам метод оптимизации нейросети (обычно лучше всего по-умолчанию работает Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

# итерируемся num_epochs раз, здесь 100000
for t in range(50000):
    # foward_pass() -- применение нейросети
    y_pred = model(X)

    # выведем loss - смотрим - ошибка на каждом этапе уменьшается
    loss = loss_fn(y_pred, Y)
    print('{} {}'.format(t, loss.data))

    # обнуляем градиенты перед backard_pass'ом (обязательно!)
    optimizer.zero_grad()

    # backward_pass() -- вычисляем градиенты loss'а по параметрам (весам) нейросети
    # этой командой мы только вычисляем градиенты, но ещё не обновляем веса
    loss.backward()

    # а тут уже обновляем веса
    optimizer.step()

In [ ]:
# Теперь можно посмотреть как можель прогнозирует
# Допустим есть вот такие параметры коробок - 1520, 1200, 1414
x = [[137.02475248, 180.57920792, 228.21287129]]
# также приводим к тензору
x = torch.autograd.Variable(torch.FloatTensor(x))
# смотрим какую плотность модель предсказала
c = model(x)
c
print(c)
